In [48]:
import  os

import pandas as pd

from bokeh.io import output_notebook, show

from bokeh.models import ColumnDataSource, GMapOptions, HoverTool, TapTool, OpenURL, PanTool, WheelZoomTool

from bokeh.plotting import gmap

API_KEY = os.environ["GOOGL_API_KEY"]

In [59]:
output_notebook()

Loading BokehJS ...

Function Calculate the plot size and location of the glyphs over the google map API

In [1]:
def ShowPricePlot(file, sheet_name):
    df = pd.read_excel(file, sheet_name=sheet_name)

    baseline = df.avg_price.min()-1
    scale = 3
    df["rad"] = df.avg_price.apply(lambda x: (x-baseline)/8)
    

    map_options = GMapOptions(lat=50.5020794, lng=-111.9912878, map_type="roadmap", zoom=3)

    p = gmap(os.environ["GOOGL_API_KEY"], map_options, tools='tap')

    source = ColumnDataSource(
        data=dict(lat=df['lat'].tolist(),
                  lon=df['lon'].tolist(),
                  rad = df['rad'].tolist(),
                  city = df['city'].tolist(),
                  avg_price = df['avg_price'].tolist(),
                  link = df['link'].tolist())
    )



    p.circle(x="lon", y="lat", size="rad", fill_color="green", fill_alpha=0.8, source=source)

    p.add_tools(PanTool(), WheelZoomTool(), HoverTool(
        tooltips=[
            # use @{ } for field names with spaces
            ( 'city', '@city'),
            ('average rate', '$@avg_price')
        ],


        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='mouse'
    ))

    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url="@link")
    return p





In [69]:
plot = ShowPricePlot("Static/CityPricesGeocoord.xlsx")

1. The size of the circle plots represent the relative average prices of the phones

2. On clicking the plots, it redirects to the CraigsList page of the respective region for the searched product

In [70]:
show(plot)